In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# Filter is the block of data (pixels) to scan an image and produce a value
filter_size = [3, 3]

# A grey scale image would have one, RGB would have 3
input_channel = 1

# Number of channels to produce. With data aggregated, each channel 
output_channel = 2

In [3]:
image_1 = [
    [[0.11], [0.12], [0.13], [0.14]],
    [[0.21], [0.22], [0.23], [0.24]],
    [[0.31], [0.32], [0.33], [0.34]],
    [[0.41], [0.42], [0.43], [0.44]]
]

input_x = [image_1]

height = len(image_1)
width = len(image_1[0])
channel = len(image_1[0][0])
print("%d x %d" % (height, width))

4 x 4


In [4]:
input_shape = [None, height, width, input_channel]
X = tf.placeholder(tf.float32, input_shape)

## Convolutional Weights and Bias
### Weight Shape: 3 x 3 x 1 x 2
    * Filter sizes: 3 x 3 (Modeler's choice)
    * Input Channels: 1 (Greyscale)
    * Output Channels: 2 (Modeler's choice)	

In [5]:
# 3 x 3 filter shape
filter1 = [
    [.4, .6, .4],
    [.6, .9, .6],
    [.1, .6, .4], 
]

# Each filter only has one input channel (grey scale)
# 3 x 3 x 1
channel_filters1 = [filter1]

# We want to output 2 channels which requires another set of 3 x 3 x 1

filter2 = [
    [.9, .5, .9],
    [.5, .3, .5],
    [.9, .5, .9],
]
channel_filters2 = [filter2]

# Initialized Weights
# 3 x 3 x 1 x 2
convolution_layer1 = [channel_filters1, channel_filters2]

print(convolution_layer1[0][0][2][0])

0.1


In [6]:
for filters in convolution_layer1:
    for channel_filter in filters:
        for row in channel_filter:
            print(row)
        print()

[0.4, 0.6, 0.4]
[0.6, 0.9, 0.6]
[0.1, 0.6, 0.4]

[0.9, 0.5, 0.9]
[0.5, 0.3, 0.5]
[0.9, 0.5, 0.9]



### Bias Shape: 2
Matches the number of output channels

In [7]:
biases_1 = [0.1, 0.1]

## Convolutional Layers

In [8]:
# Number of pixels to shift want evaluating a filter
stride_1 = 1

# Transpose to match inputs 
W1 = tf.Variable(np.transpose(convolution_layer1), dtype=tf.float32)
B1 = tf.Variable(biases_1, dtype=tf.float32)
print(W1.shape)

(3, 3, 1, 2)


# Activation Shape: 4 x 4 x 2
* dimension_1 = hight / stride
* dimension_2 = width / stride
* dimension_3 = output_channel

In [9]:
stride_shape = [1, stride_1, stride_1, 1]
preactivation = tf.nn.conv2d(X, W1, strides=stride_shape, padding='SAME') + B1
activation_1 = tf.nn.relu(preactivation)

print(activation_1.shape)

(?, 4, 4, 2)


In [10]:
# Create a session
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

In [11]:
x = sess.run(W1)

# Transpose to match our model 
print(np.transpose(x))

[[[[ 0.40000001  0.60000002  0.40000001]
   [ 0.60000002  0.89999998  0.60000002]
   [ 0.1         0.60000002  0.40000001]]]


 [[[ 0.89999998  0.5         0.89999998]
   [ 0.5         0.30000001  0.5       ]
   [ 0.89999998  0.5         0.89999998]]]]


In [12]:
# Transpose to match our model 
feed_dict = {X: input_x}
Y1 = activation_1.eval(session=sess, feed_dict=feed_dict)

print(np.transpose(Y1))

[[[[ 0.48499998]
   [ 0.81300002]
   [ 1.13300002]
   [ 0.93900007]]

  [[ 0.66000003]
   [ 1.13900006]
   [ 1.5990001 ]
   [ 1.33100009]]

  [[ 0.69500005]
   [ 1.18500006]
   [ 1.64499998]
   [ 1.36300004]]

  [[ 0.54000002]
   [ 0.92600006]
   [ 1.27600002]
   [ 1.09000003]]]


 [[[ 0.49599996]
   [ 0.87900001]
   [ 1.23899996]
   [ 0.87600005]]

  [[ 0.76200002]
   [ 1.39799988]
   [ 1.98799992]
   [ 1.38199997]]

  [[ 0.79799998]
   [ 1.45700002]
   [ 2.04699993]
   [ 1.41799998]]

  [[ 0.53400004]
   [ 0.94100004]
   [ 1.301     ]
   [ 0.91400003]]]]


# Activation2 Shape: 2 x 2 x 4
* dimension_1 = d1 / stride
* dimension_2 = d2 / stride
* dimension_3 = output_channel

In [13]:
init_2 = tf.truncated_normal([4, 4, 2, 4], stddev=0.1)
W2 = tf.Variable(init_2)
B2 = tf.Variable(tf.ones([4])/10)

stride_2 = 2


In [16]:
strides = [1, stride_2, stride_2, 1]
preactivate = tf.nn.conv2d(activation_1, W2, strides=strides, padding='SAME') + B2
activation_2 = tf.nn.relu(preactivate)

print(activation_2.shape)

(2, 2, 4)


In [22]:
# reshape the output from the third convolution for the fully connected layer
# -1 x 2 x 2 x 4
re_shape = [-1, np.multiply.reduce(list(activation_2.shape[1:]))]

YY = tf.reshape(activation_2, shape=re_shape)

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [-1, Dimension(16)]. Consider casting elements to a supported type.